# A full business solution

## Now we will take our project from Day 1 to the next level

### BUSINESS CHALLENGE:

Create a product that builds a Brochure for a company to be used for prospective clients, investors and potential recruits.

We will be provided a company name and their primary website.

See the end of this notebook for examples of real-world business applications.

And remember: I'm always available if you have problems or ideas! Please do reach out.

In [3]:
# imports
# If these fail, please check you're running from an 'activated' environment with (llms) in the command prompt

import os
import requests
import json
from typing import List
from dotenv import load_dotenv
from bs4 import BeautifulSoup
from IPython.display import Markdown, display, update_display
from openai import OpenAI

In [4]:
# Initialize and constants

load_dotenv(override=True)
api_key = os.getenv('OPENAI_API_KEY')

if api_key and api_key.startswith('sk-proj-') and len(api_key)>10:
    print("API key looks good so far")
else:
    print("There might be a problem with your API key? Please visit the troubleshooting notebook!")
    
MODEL = 'gpt-4o-mini'
openai = OpenAI()

API key looks good so far


In [5]:
# A class to represent a Webpage

# Some websites need you to use proper headers when fetching them:
headers = {
 "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/117.0.0.0 Safari/537.36"
}

class Website:                           
    """
    A utility class to represent a Website that we have scraped, now with links
    """

    def __init__(self, url):
        self.url = url
        response = requests.get(url, headers=headers)
        self.body = response.content
        soup = BeautifulSoup(self.body, 'html.parser')
        self.title = soup.title.string if soup.title else "No title found"
        if soup.body:
            for irrelevant in soup.body(["script", "style", "img", "input"]):
                irrelevant.decompose()
            self.text = soup.body.get_text(separator="\n", strip=True)
        else:
            self.text = ""
        links = [link.get('href') for link in soup.find_all('a')]
        self.links = [link for link in links if link]

    def get_contents(self):
        return f"Webpage Title:\n{self.title}\nWebpage Contents:\n{self.text}\n\n"

In [6]:
ed = Website("https://soaapp.github.io")
ed.links

['#about',
 '#experience',
 '#contact',
 '/files/JayJ_Resume2025_SEv1.pdf',
 '#experience',
 'mailto:jayjahanzad@gmail.com',
 'https://github.com/soaapp',
 'https://linkedin.com/in/jayjahanzad',
 'mailto:jayjahanzad@gmail.com']

## First step: Have GPT-4o-mini figure out which links are relevant

### Use a call to gpt-4o-mini to read the links on a webpage, and respond in structured JSON.  
It should decide which links are relevant, and replace relative links such as "/about" with "https://company.com/about".  
We will use "one shot prompting" in which we provide an example of how it should respond in the prompt.

This is an excellent use case for an LLM, because it requires nuanced understanding. Imagine trying to code this without LLMs by parsing and analyzing the webpage - it would be very hard!

Sidenote: there is a more advanced technique called "Structured Outputs" in which we require the model to respond according to a spec. We cover this technique in Week 8 during our autonomous Agentic AI project.

In [7]:
link_system_prompt = "You are provided with a list of links found on a webpage. \
You are able to decide which of the links would be most relevant to include in a brochure about the company, \
such as links to an About page, or a Company page, or Careers/Jobs pages.\n"
link_system_prompt += "You should respond in JSON as in this example:"
link_system_prompt += """
{
    "links": [
        {"type": "about page", "url": "https://full.url/goes/here/about"},
        {"type": "careers page": "url": "https://another.full.url/careers"}
    ]
}
"""

In [8]:
print(link_system_prompt)

You are provided with a list of links found on a webpage. You are able to decide which of the links would be most relevant to include in a brochure about the company, such as links to an About page, or a Company page, or Careers/Jobs pages.
You should respond in JSON as in this example:
{
    "links": [
        {"type": "about page", "url": "https://full.url/goes/here/about"},
        {"type": "careers page": "url": "https://another.full.url/careers"}
    ]
}



In [9]:
def get_links_user_prompt(website):
    user_prompt = f"Here is the list of links on the website of {website.url} - "
    user_prompt += "please decide which of these are relevant web links for a brochure about the company, respond with the full https URL in JSON format. \
Do not include Terms of Service, Privacy, email links.\n"
    user_prompt += "Links (some might be relative links):\n"
    user_prompt += "\n".join(website.links)
    return user_prompt

In [10]:
print(get_links_user_prompt(ed))

Here is the list of links on the website of https://soaapp.github.io - please decide which of these are relevant web links for a brochure about the company, respond with the full https URL in JSON format. Do not include Terms of Service, Privacy, email links.
Links (some might be relative links):
#about
#experience
#contact
/files/JayJ_Resume2025_SEv1.pdf
#experience
mailto:jayjahanzad@gmail.com
https://github.com/soaapp
https://linkedin.com/in/jayjahanzad
mailto:jayjahanzad@gmail.com


In [11]:
def get_links(url):
    website = Website(url)
    response = openai.chat.completions.create(
        model=MODEL,
        messages=[
            {"role": "system", "content": link_system_prompt},
            {"role": "user", "content": get_links_user_prompt(website)}
      ],
        response_format={"type": "json_object"}
    )
    result = response.choices[0].message.content
    return json.loads(result)

In [12]:
# Anthropic has made their site harder to scrape, so I'm using HuggingFace..

huggingface = Website("https://huggingface.co")
huggingface.links

['/',
 '/models',
 '/datasets',
 '/spaces',
 '/posts',
 '/docs',
 '/enterprise',
 '/pricing',
 '/login',
 '/join',
 '/blog/inference-providers',
 '/spaces',
 '/models',
 '/deepseek-ai/DeepSeek-R1',
 '/Zyphra/Zonos-v0.1-hybrid',
 '/agentica-org/DeepScaleR-1.5B-Preview',
 '/Zyphra/Zonos-v0.1-transformer',
 '/hexgrad/Kokoro-82M',
 '/models',
 '/spaces/black-forest-labs/FLUX.1-dev',
 '/spaces/tencent/Hunyuan3D-2',
 '/spaces/deepseek-ai/Janus-Pro-7B',
 '/spaces/m-ric/open_Deep-Research',
 '/spaces/lllyasviel/LuminaBrush',
 '/spaces',
 '/datasets/open-r1/OpenR1-Math-220k',
 '/datasets/open-thoughts/OpenThoughts-114k',
 '/datasets/saiyan-world/Goku-MovieGenBench',
 '/datasets/fka/awesome-chatgpt-prompts',
 '/datasets/Anthropic/EconomicIndex',
 '/datasets',
 '/join',
 '/pricing#endpoints',
 '/pricing#spaces',
 '/pricing',
 '/enterprise',
 '/enterprise',
 '/enterprise',
 '/enterprise',
 '/enterprise',
 '/enterprise',
 '/enterprise',
 '/allenai',
 '/facebook',
 '/amazon',
 '/google',
 '/Intel',


In [13]:
get_links("https://huggingface.co")

{'links': [{'type': 'about page', 'url': 'https://huggingface.co'},
  {'type': 'careers page', 'url': 'https://apply.workable.com/huggingface/'},
  {'type': 'enterprise page', 'url': 'https://huggingface.co/enterprise'},
  {'type': 'blog page', 'url': 'https://huggingface.co/blog'},
  {'type': 'join page', 'url': 'https://huggingface.co/join'},
  {'type': 'contact page', 'url': 'https://huggingface.co/contact'},
  {'type': 'GitHub page', 'url': 'https://github.com/huggingface'},
  {'type': 'Twitter page', 'url': 'https://twitter.com/huggingface'},
  {'type': 'LinkedIn page',
   'url': 'https://www.linkedin.com/company/huggingface/'}]}

## Second step: make the brochure!

Assemble all the details into another prompt to GPT4-o

In [14]:
def get_all_details(url):
    result = "Landing page:\n"
    result += Website(url).get_contents()
    links = get_links(url)
    print("Found links:", links)
    for link in links["links"]:
        result += f"\n\n{link['type']}\n"
        result += Website(link["url"]).get_contents()
    return result

In [15]:
print(get_all_details("https://huggingface.co"))

Found links: {'links': [{'type': 'about page', 'url': 'https://huggingface.co/huggingface'}, {'type': 'careers page', 'url': 'https://apply.workable.com/huggingface/'}, {'type': 'enterprise page', 'url': 'https://huggingface.co/enterprise'}, {'type': 'pricing page', 'url': 'https://huggingface.co/pricing'}, {'type': 'blog page', 'url': 'https://huggingface.co/blog'}, {'type': 'docs page', 'url': 'https://huggingface.co/docs'}, {'type': 'discussion forum', 'url': 'https://discuss.huggingface.co'}, {'type': 'GitHub page', 'url': 'https://github.com/huggingface'}, {'type': 'LinkedIn page', 'url': 'https://www.linkedin.com/company/huggingface/'}, {'type': 'Twitter page', 'url': 'https://twitter.com/huggingface'}]}
Landing page:
Webpage Title:
Hugging Face – The AI community building the future.
Webpage Contents:
Hugging Face
Models
Datasets
Spaces
Posts
Docs
Enterprise
Pricing
Log In
Sign Up
NEW
Welcome to Inference Providers on the Hub 🔥
smolagents - a smol library to build great agents
U

In [16]:
system_prompt = "You are an assistant that analyzes the contents of several relevant pages from a company website \
and creates a short brochure about the company for prospective customers, investors and recruits. Respond in markdown.\
Include details of company culture, customers and careers/jobs if you have the information."

# Or uncomment the lines below for a more humorous brochure - this demonstrates how easy it is to incorporate 'tone':

# system_prompt = "You are an assistant that analyzes the contents of several relevant pages from a company website \
# and creates a short humorous, entertaining, jokey brochure about the company for prospective customers, investors and recruits. Respond in markdown.\
# Include details of company culture, customers and careers/jobs if you have the information."


In [17]:
def get_brochure_user_prompt(company_name, url):
    user_prompt = f"You are looking at a company called: {company_name}\n"
    user_prompt += f"Here are the contents of its landing page and other relevant pages; use this information to build a short brochure of the company in markdown.\n"
    user_prompt += get_all_details(url)
    user_prompt = user_prompt[:5_000] # Truncate if more than 5,000 characters
    return user_prompt

In [18]:
get_brochure_user_prompt("HuggingFace", "https://huggingface.co")

Found links: {'links': [{'type': 'about page', 'url': 'https://huggingface.co/huggingface'}, {'type': 'careers page', 'url': 'https://apply.workable.com/huggingface/'}, {'type': 'blog', 'url': 'https://huggingface.co/blog'}, {'type': 'enterprise', 'url': 'https://huggingface.co/enterprise'}, {'type': 'pricing', 'url': 'https://huggingface.co/pricing'}, {'type': 'models', 'url': 'https://huggingface.co/models'}, {'type': 'datasets', 'url': 'https://huggingface.co/datasets'}, {'type': 'spaces', 'url': 'https://huggingface.co/spaces'}]}


"You are looking at a company called: HuggingFace\nHere are the contents of its landing page and other relevant pages; use this information to build a short brochure of the company in markdown.\nLanding page:\nWebpage Title:\nHugging Face – The AI community building the future.\nWebpage Contents:\nHugging Face\nModels\nDatasets\nSpaces\nPosts\nDocs\nEnterprise\nPricing\nLog In\nSign Up\nNEW\nWelcome to Inference Providers on the Hub 🔥\nsmolagents - a smol library to build great agents\nUse models from the HF Hub in LM Studio\nThe AI community building the future.\nThe platform where the machine learning community collaborates on models, datasets, and applications.\nExplore AI Apps\nor\nBrowse 1M+ models\nTrending on\nthis week\nModels\ndeepseek-ai/DeepSeek-R1\nUpdated\n6 days ago\n•\n3.88M\n•\n8.96k\nZyphra/Zonos-v0.1-hybrid\nUpdated\nabout 6 hours ago\n•\n9.36k\n•\n787\nagentica-org/DeepScaleR-1.5B-Preview\nUpdated\n5 days ago\n•\n7.81k\n•\n336\nZyphra/Zonos-v0.1-transformer\nUpdated\

In [19]:
def create_brochure(company_name, url):
    response = openai.chat.completions.create(
        model=MODEL,
        messages=[
            {"role": "system", "content": system_prompt},
            {"role": "user", "content": get_brochure_user_prompt(company_name, url)}
          ],
    )
    result = response.choices[0].message.content
    display(Markdown(result))

In [20]:
create_brochure("HuggingFace", "https://huggingface.co")

Found links: {'links': [{'type': 'about page', 'url': 'https://huggingface.co'}, {'type': 'enterprise page', 'url': 'https://huggingface.co/enterprise'}, {'type': 'pricing page', 'url': 'https://huggingface.co/pricing'}, {'type': 'careers page', 'url': 'https://apply.workable.com/huggingface/'}, {'type': 'blog page', 'url': 'https://huggingface.co/blog'}, {'type': 'Twitter page', 'url': 'https://twitter.com/huggingface'}, {'type': 'LinkedIn page', 'url': 'https://www.linkedin.com/company/huggingface/'}]}


```markdown
# Welcome to Hugging Face

**Hugging Face - The AI community building the future.**

At Hugging Face, we believe in the power of collaboration and innovation in the realm of AI and machine learning (ML). Our platform serves as a pivotal hub for the global ML community, driving advancements through shared models, datasets, and applications. 

## About Us

Founded with a mission to democratize AI, Hugging Face is dedicated to building open-source tools that promote collaboration amongst developers, researchers, and enthusiasts. With over 1 million models and 250,000 datasets to explore, we empower everyone from small startups to large enterprises to harness the power of machine learning.

## Our Products

- **Models**: A diverse assortment of over a million high-quality ML models, including the latest advancements in natural language processing and image recognition.
- **Datasets**: Access to a vast library of datasets for various tasks including NLP, computer vision, and audio analysis.
- **Spaces**: Interactive applications that allow users to experiment and create using cutting-edge technology.
- **Inference Endpoints**: Deploy optimized endpoints for seamless integration of AI applications.

## Our Customers

Our solutions are trusted by more than 50,000 organizations, including notable names such as:
- **Meta**
- **Amazon Web Services**
- **Google**
- **Microsoft**

Whether you are a startup looking for AI solutions or a multinational enterprise requiring advanced security and performance tools, Hugging Face has something for everyone.

## Company Culture

At Hugging Face, we foster a community-centric environment that encourages collaboration, creativity, and continuous learning. Our culture is built around the following core values:
- **Openness**: We believe in transparency and sharing knowledge within the community.
- **Sustainability**: Our practices aim to promote long-term issues in AI and its societal implications.
- **Innovation**: Embracing new ideas and pushing boundaries to redefine what is possible in AI.

## Careers at Hugging Face

Join us and be part of a passionate team pushing the limits of technology! We are always looking for talented individuals who are excited about building the future of AI. Current job openings can be found on our [Careers Page](https://huggingface.co/careers).

### Why Work with Us?
- **Dynamic Work Environment**: Work alongside industry leaders and driven professionals.
- **Impact**: Be part of projects that make a difference in the world of technology.
- **Growth Opportunities**: Access to continuous training, conferences, and resources to enhance your skills.

## Join Us Today!

Explore our AI applications, discover our rich library of models and datasets, or sign up for our services to accelerate your machine learning projects.

Hugging Face is not just a platform; it’s a thriving community. Join us in building the future of AI.

For more information:
- [Website](https://huggingface.co)
- [Sign Up](https://huggingface.co/sign-up)

Follow us on our social channels:
- [Twitter](https://twitter.com/huggingface)
- [LinkedIn](https://linkedin.com/company/huggingface)
- [Discord](https://discord.gg/huggingface)

---

*Together, let's shape the future of machine learning!*
```

## Finally - a minor improvement

With a small adjustment, we can change this so that the results stream back from OpenAI,
with the familiar typewriter animation

In [22]:
def stream_brochure(company_name, url):
    stream = openai.chat.completions.create(
        model=MODEL,
        messages=[
            {"role": "system", "content": system_prompt},
            {"role": "user", "content": get_brochure_user_prompt(company_name, url)}
          ],
        stream=True
    )
    
    response = ""
    display_handle = display(Markdown(""), display_id=True)
    for chunk in stream:
        response += chunk.choices[0].delta.content or ''
        response = response.replace("```","").replace("markdown", "")
        update_display(Markdown(response), display_id=display_handle.display_id)

In [23]:
stream_brochure("HuggingFace", "https://huggingface.co")

Found links: {'links': [{'type': 'company page', 'url': 'https://huggingface.co'}, {'type': 'about page', 'url': 'https://huggingface.co/huggingface'}, {'type': 'careers page', 'url': 'https://apply.workable.com/huggingface/'}, {'type': 'blog', 'url': 'https://huggingface.co/blog'}, {'type': 'enterprise page', 'url': 'https://huggingface.co/enterprise'}, {'type': 'pricing page', 'url': 'https://huggingface.co/pricing'}, {'type': 'join page', 'url': 'https://huggingface.co/join'}]}


# Hugging Face Company Brochure

## Welcome to Hugging Face
### The AI Community Building the Future

At Hugging Face, we are at the forefront of the artificial intelligence revolution. Our mission is to create a collaborative platform where the machine learning community can come together to innovate with models, datasets, and applications. We believe AI has the power to transform the future, and we invite you to join us on this exciting journey.

---

## Our Offerings

### Models and Datasets
- **Explore Over 1 Million Models**: From deep learning to reinforcement learning, we host an extensive collection of state-of-the-art machine learning models on our platform.
- **250,000+ Datasets**: Access a vast array of datasets for various tasks in computer vision, NLP, and audio processing.

### Spaces and Inference
- **Collaborate and Share**: Run your applications in our collaborative spaces, whether it’s for image generation, text analysis, or 3D transformations.
- **Optimized Inference Endpoints**: We provide compute solutions that are easy to scale and deploy.

### Solutions for Enterprises
- **Customized Solutions**: Hugging Face offers enterprise-level solutions with robust security, priority support, and dedicated resources for teams.
- **Flexible Pricing**: Starting as low as $20 per user per month for teams to access our platform.

---

## Company Culture

At Hugging Face, our culture thrives on:
- **Collaboration**: We foster a community-oriented approach, where ideas and projects flourish through teamwork.
- **Innovation**: We are committed to pushing the boundaries of what’s possible in AI through continuous research and development.
- **Inclusivity**: Emphasizing a diverse work environment where all voices are heard and valued.

Our team includes passionate individuals from various backgrounds who share a common vision for the future of AI.

---

## Join Us

### Career Opportunities
We’re always looking for talented individuals to join our growing team. Whether you’re interested in research, engineering, or contributing to our community projects, there’s a place for you at Hugging Face.

**Current Openings Include:**
- Machine Learning Engineer
- Data Scientist
- Community Manager
- Software Developer

Visit our [Careers](https://huggingface.co/jobs) page to explore the current listings and find your opportunity to be a part of our mission.

---

## Our Customers
We are proud to serve over 50,000 organizations, including industry giants such as:
- **Meta**
- **Amazon Web Services**
- **Microsoft**
- **Google**

These partnerships enable us to continuously enhance our offerings and support a global AI community.

---

## Get Involved
Whether you're an AI novice or an expert, Hugging Face is your portal to a vibrant machine learning ecosystem.  
Explore our resources, collaborate on interesting projects, or leverage our tools to build your own AI applications.

**Join us at Hugging Face in shaping the future of AI.**

Connect with us on:
- [Twitter](https://twitter.com/huggingface)
- [GitHub](https://github.com/huggingface)
- [LinkedIn](https://linkedin.com/company/huggingface)
- [Discord](https://discord.com/invite/huggingface)

---

Embrace the AI future with Hugging Face – where collaboration meets innovation!

In [24]:
# Try changing the system prompt to the humorous version when you make the Brochure for Hugging Face:

stream_brochure("HuggingFace", "https://huggingface.co")

Found links: {'links': [{'type': 'about page', 'url': 'https://huggingface.co/huggingface'}, {'type': 'careers page', 'url': 'https://apply.workable.com/huggingface/'}, {'type': 'enterprise page', 'url': 'https://huggingface.co/enterprise'}, {'type': 'pricing page', 'url': 'https://huggingface.co/pricing'}, {'type': 'blog page', 'url': 'https://huggingface.co/blog'}, {'type': 'community page', 'url': 'https://discuss.huggingface.co'}, {'type': 'GitHub page', 'url': 'https://github.com/huggingface'}, {'type': 'Twitter page', 'url': 'https://twitter.com/huggingface'}, {'type': 'LinkedIn page', 'url': 'https://www.linkedin.com/company/huggingface/'}]}


# Hugging Face Brochure

## Welcome to Hugging Face

**The AI community building the future.**

At Hugging Face, we are dedicated to fostering an open and collaborative environment for the machine learning community. With an extensive library of models, datasets, and innovative applications, our platform enables users from various backgrounds to explore, create, and share cutting-edge artificial intelligence solutions.

---

## Company Culture

Hugging Face thrives on community collaboration and is committed to open-source principles. We believe in inclusivity and encouraging contributions from everyone, whether you're a seasoned machine learning expert or just starting on your AI journey. Our culture emphasizes continuous learning, experimentation, and sharing knowledge. Together, we’re building the future of AI by ensuring that tools and resources are accessible to all.

---

## Our Offerings

- **Models**: Explore over **1 million models** for a variety of tasks in text, image, audio, and even 3D. Each model is hosted for collaboration and discovery.
- **Datasets**: Browse through **250,000+ datasets** available for a range of applications in machine learning.
- **Spaces**: Use **Hugging Face Spaces** to host your machine learning apps. Our tools support running models on powerful environments, enhancing real-time applications.

### Enterprise Solutions

For organizations, we offer **Enterprise** solutions tailored for advanced AI development with features like dedicated support, security, and access controls. Join over **50,000 organizations** already leveraging Hugging Face to power their AI initiatives.

---

## Our Customers

We proudly serve a diverse range of customers, including iconic tech leaders:

- **Meta**: 2,060+ models
- **Amazon Web Services**: 20 models
- **Google**: 913 models
- **Microsoft**: 355 models
- and many more…

With **advanced tools** and resources, we empower organizations to harness the full potential of AI technology.

---

## Careers at Hugging Face

Join a passionate team that’s at the forefront of AI innovation. We are constantly seeking talented individuals who share our commitment to open-source technology and community building. Explore career opportunities where you can contribute directly to projects that shape the future of machine learning.

---

## Start Your Journey Today

Whether you are looking to enhance your skills, share your projects, or develop enterprise-level AI solutions, **Hugging Face** is the right place for you. 

- **Sign Up**: Create your account and begin exploring our resources.
- **Join the Community**: Contribute to open-source projects and collaborate with other innovators in the field.

**Join us and let's build a brighter AI future together!**

---

## Connect with Us

Stay up-to-date with the latest developments in AI and Hugging Face by following us on:

- [GitHub](https://github.com/huggingface)
- [Twitter](https://twitter.com/huggingface)
- [LinkedIn](https://www.linkedin.com/company/hugging-face)
- [Discord](https://discord.com/invite/huggingface)
  
For more information, visit our official website: [Hugging Face](https://huggingface.co)

---

**Hugging Face - The AI Community Building the Future.**

<table style="margin: 0; text-align: left;">
    <tr>
        <td style="width: 150px; height: 150px; vertical-align: middle;">
            <img src="../business.jpg" width="150" height="150" style="display: block;" />
        </td>
        <td>
            <h2 style="color:#181;">Business applications</h2>
            <span style="color:#181;">In this exercise we extended the Day 1 code to make multiple LLM calls, and generate a document.

This is perhaps the first example of Agentic AI design patterns, as we combined multiple calls to LLMs. This will feature more in Week 2, and then we will return to Agentic AI in a big way in Week 8 when we build a fully autonomous Agent solution.

Generating content in this way is one of the very most common Use Cases. As with summarization, this can be applied to any business vertical. Write marketing content, generate a product tutorial from a spec, create personalized email content, and so much more. Explore how you can apply content generation to your business, and try making yourself a proof-of-concept prototype.</span>
        </td>
    </tr>
</table>

<table style="margin: 0; text-align: left;">
    <tr>
        <td style="width: 150px; height: 150px; vertical-align: middle;">
            <img src="../important.jpg" width="150" height="150" style="display: block;" />
        </td>
        <td>
            <h2 style="color:#900;">Before you move to Week 2 (which is tons of fun)</h2>
            <span style="color:#900;">Please see the week1 EXERCISE notebook for your challenge for the end of week 1. This will give you some essential practice working with Frontier APIs, and prepare you well for Week 2.</span>
        </td>
    </tr>
</table>

<table style="margin: 0; text-align: left;">
    <tr>
        <td style="width: 150px; height: 150px; vertical-align: middle;">
            <img src="../resources.jpg" width="150" height="150" style="display: block;" />
        </td>
        <td>
            <h2 style="color:#f71;">A reminder on 2 useful resources</h2>
            <span style="color:#f71;">1. The resources for the course are available <a href="https://edwarddonner.com/2024/11/13/llm-engineering-resources/">here.</a><br/>
            2. I'm on LinkedIn <a href="https://www.linkedin.com/in/eddonner/">here</a> and I love connecting with people taking the course!
            </span>
        </td>
    </tr>
</table>

<table style="margin: 0; text-align: left;">
    <tr>
        <td style="width: 150px; height: 150px; vertical-align: middle;">
            <img src="../thankyou.jpg" width="150" height="150" style="display: block;" />
        </td>
        <td>
            <h2 style="color:#090;">Finally! I have a special request for you</h2>
            <span style="color:#090;">
                My editor tells me that it makes a MASSIVE difference when students rate this course on Udemy - it's one of the main ways that Udemy decides whether to show it to others. If you're able to take a minute to rate this, I'd be so very grateful! And regardless - always please reach out to me at ed@edwarddonner.com if I can help at any point.
            </span>
        </td>
    </tr>
</table>